### Evaluating and Improving the model

In [26]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values


from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer


# Male/Female
le_X = LabelEncoder()
X[:, 2] = le_X.fit_transform(X[:, 2])

# Country column
ct = ColumnTransformer([("Country", OneHotEncoder(), [1])], remainder = 'passthrough')
X = ct.fit_transform(X)



X = X[:,1:]


# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


In [2]:
# Fitting classifier to the Training set
# Create your classifier here
import keras

#sequential model to initialize the neural networks and dense module to build the layers for NN
from keras.models import Sequential
from keras.layers import Dense

"""
# two ways to build a deep learning model
# 1. defining sequence of layers for NN
# 2. Or Defining the layers in the form of Graph
"""

#initializing the ANN
cls = Sequential()

"""
# adding the first layer, and input layers also our first hidden layer
# output_dim - number of nodes in the layer, can be average of input and output dim
# init - weigts to be initilized
# activation - activtion function you want to use in the layer
"""

# adding input and first hidden layer
cls.add(Dense(output_dim = 6,init = 'uniform',activation = 'relu',input_dim = 11))

# adding second hidden layer
cls.add(Dense(output_dim = 6,init = 'uniform',activation = 'relu'))

# adding third hidden layer
cls.add(Dense(output_dim = 6,init = 'uniform',activation = 'relu'))


# adding output layer
"""
if we have more than one dependent variables, 
whihc might be the case when we do one hot encoding for multi class dependent variable.
We should pick the out_dim based on number of varibales we have and our activation will be 'softmax'
"""
cls.add(Dense(output_dim = 1,init = 'uniform',activation = 'sigmoid'))


# compiling the ANN
"""
optimizer = algorithm to optimize for the weights (GD,SGD,MGD)
            adam is one of the SGD available
loss = algorithm used to calculate the cost or loss between the actual vs predicted
        log loss is used for binary classification (binary_crossentropy for binary)
metrics = metric used to evaluate your model
"""
cls.compile(optimizer = 'adam',
            loss = 'binary_crossentropy',
            metrics = ['accuracy'])

# fitting the ANN on our training data
"""
X - your training independent varibales data
y - your training dependent varibal data
batch_size - decides whether we are doing the GD, SD or MGD
nb_epochs - number of times the 
"""
cls.fit(X_train,y_train,batch_size = 10,nb_epoch = 100)


from sklearn.metrics import confusion_matrix


def get_confMatrix(y_pred,y_test,threshold):
    y_pred = [1 if y >=threshold else 0 for y in y_pred] 
    # Making the Confusion Matrix
    return confusion_matrix(y_test, y_pred)

# Predicting the Test set results
y_pred = cls.predict(X_test)

cm = get_confMatrix(y_pred,y_test,0.5)

print("accuracy is {}".format((cm[0,0]+cm[1,1])/len(y_test)))
print("precision is {}".format((cm[1,1])/(cm[1,1]+cm[0,1])))
print("recall is {}".format((cm[1,1])/(cm[1,0]+cm[1,1])))


Using TensorFlow backend.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=11, units=6, kernel_initializer="uniform")`
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
7500/7500 [==============================] - 1s 119us/step - loss: 0.4841 - accuracy: 0.7961
Epoch 2/100
7500/7500 [==============================] - 1s 87us/step - loss: 0.4297 - accuracy: 0.7963
Epoch 3/100
7500/7500 [==============================] - 1s 85us/step - loss: 0.4231 - accuracy: 0.7963
Epoch 4/100
7500/7500 [==============================] - 1s 87us/step - loss: 0.4189 - accuracy: 0.8213
Epoch 5/100
7500/7500 [==============================] - 1s 85us/step - loss: 0.4164 - accuracy: 0.8284
Epoch 6/100
7500/7500 [==============================] - 1s 86us/step - loss: 0.4145 - accuracy: 0.8296
Epoch 7/100
7500/7500 [==============================] - 1s 85us/step - loss: 0.4138 - accuracy: 0.8304
Epoch 8/100
7500/7500 [==============================] - 1s 85us/step - loss: 0.4125 - accuracy: 0.8317
Epoch 9/100
7500/7500 [==============================] - 1s 83us/step - loss: 0.4115 - accuracy: 0.8313
Epoch 10/100
7500/7500 [==============================] - 1s 88

#### K fold Crossvalidation

In [40]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

"""
The keras wrappers lib has scikit_learn cross validation function in the form of kerasClassifier
kerasClassfier takes in a functions as input, whihc is nothing but your ANN classfier layers in the form of a function

"""

def build_churn_classifier():
    cls = Sequential()
    cls.add(Dense(output_dim = 6, init = 'uniform',activation = 'relu',input_dim = 11))
    cls.add(Dense(output_dim = 6, init = 'uniform',activation = 'relu'))
    cls.add(Dense(output_dim = 1, init = 'uniform',activation = 'sigmoid'))
    cls.compile(optimizer = 'adam',
               loss = 'binary_crossentropy',
               metrics = ['accuracy'])
    return cls
cls = KerasClassifier(build_fn = build_churn_classifier,batch_size = 10,epochs = 100)
cv_scores = cross_val_score(cls,X = X_train,y = y_train,cv = 10,n_jobs = -1)

print("Our Cross validation mean and variance score is {}%,{}% respectively".format(
                                                                                np.round(cv_scores.mean()*100,2),
                                                                                np.round(cv_scores.std()*100,2)))

Our Cross validation mean and variance score is 83.72%,1.27% respectively


### dropout regularization -- to reduce overfitting

In [36]:

"""
In dropout regularization in every iteration randomly some of the neurons are randomly disabled,
so that our ANN is not over relying on a particular neuron, this will make sure of two things
1. Every neuron is independently try to workout a relationship without relying on other 
2. Also We dont have a single configuration at every itteration 
which will make sure overfitting is avoided
"""

from keras.layers import Dropout

#initializing the ANN
cls = Sequential()

# adding input and first hidden layer without dropout function for disabling 
cls.add(Dense(output_dim = 6,init = 'uniform',activation = 'relu',input_dim = 11))
cls.add(Dropout(p=0.1))


# adding second hidden layer with dropout fn
cls.add(Dense(output_dim = 6,init = 'uniform',activation = 'relu'))
cls.add(Dropout(p=0.1))

# adding third hidden layer with dropout function
cls.add(Dense(output_dim = 6,init = 'uniform',activation = 'relu'))
cls.add(Dropout(p=0.1))

# adding output layer
cls.add(Dense(output_dim = 1,init = 'uniform',activation = 'sigmoid'))


# compiling the ANN
cls.compile(optimizer = 'adam',
            loss = 'binary_crossentropy',
            metrics = ['accuracy'])

# fitting the ANN on our training data
cls.fit(X_train,y_train,batch_size = 10,nb_epoch = 100)

def get_confMatrix(y_pred,y_test,threshold):
    y_pred = [1 if y >=threshold else 0 for y in y_pred] 
    # Making the Confusion Matrix
    return confusion_matrix(y_test, y_pred)

# Predicting the Test set results
y_pred = cls.predict(X_test)

cm = get_confMatrix(y_pred,y_test,0.5)

print("accuracy is {}".format((cm[0,0]+cm[1,1])/len(y_test)))
print("precision is {}".format((cm[1,1])/(cm[1,1]+cm[0,1])))
print("recall is {}".format((cm[1,1])/(cm[1,0]+cm[1,1])))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=11, units=6, kernel_initializer="uniform")`
  from ipykernel import kernelapp as app
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  app.launch_new_instance()
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
/opt/anaconda3/lib/python3.7/site-packag

Epoch 1/100
8000/8000 [==============================] - 1s 116us/step - loss: 0.5012 - accuracy: 0.7950
Epoch 2/100
8000/8000 [==============================] - 1s 95us/step - loss: 0.4481 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 94us/step - loss: 0.4417 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 95us/step - loss: 0.4434 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 1s 89us/step - loss: 0.4343 - accuracy: 0.7960
Epoch 6/100
8000/8000 [==============================] - 1s 104us/step - loss: 0.4351 - accuracy: 0.7960
Epoch 7/100
8000/8000 [==============================] - 1s 102us/step - loss: 0.4345 - accuracy: 0.7960
Epoch 8/100
8000/8000 [==============================] - 1s 101us/step - loss: 0.4356 - accuracy: 0.7960
Epoch 9/100
8000/8000 [==============================] - 1s 99us/step - loss: 0.4317 - accuracy: 0.7960
Epoch 10/100
8000/8000 [==============================] - 1s